# Ansible For Developers Demo

Demonstration Notebook for https://demo.ukr.lk

## Checking the inventory

### Listing the inventory

In [7]:
!ansible-inventory -i inventory.yaml --list

{
    "_meta": {
        "hostvars": {
            "vm01": {
                "ansible_host": "demo1.ukr.lk",
                "ansible_user": "root"
            }
        }
    },
    "all": {
        "children": [
            "ungrouped",
            "webservers"
        ]
    },
    "webservers": {
        "hosts": [
            "vm01"
        ]
    }
}


### Ping the hosts on the inventory

In [8]:
!ansible webservers -m ping -i inventory.yaml

vm01 | SUCCESS => {
    "ansible_facts": {
        "discovered_interpreter_python": "/usr/bin/python3"
    },
    "changed": false,
    "ping": "pong"
}


## Running Playbooks

### Run basic playbook

In [9]:
!ansible-playbook -i inventory.yaml playbook_1.yaml


PLAY [Configure Servers] *******************************************************

TASK [Gathering Facts] *********************************************************
ok: [vm01]

TASK [Apt cache is updated] ****************************************************
changed: [vm01]

TASK [Custom MOTD is set] ******************************************************
changed: [vm01]

PLAY RECAP *********************************************************************
vm01                       : ok=3    changed=2    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



### Run playbook with basic web server setup

In [1]:
!ansible-playbook -i inventory.yaml playbook_2.yaml


PLAY [Configure Servers] *******************************************************

TASK [Gathering Facts] *********************************************************
ok: [vm01]

TASK [Apt cache is updated] ****************************************************
ok: [vm01]

TASK [Custom MOTD is set] ******************************************************
ok: [vm01]

PLAY [Configure Webservers] ****************************************************

TASK [Gathering Facts] *********************************************************
ok: [vm01]

TASK [Latest nginx is installed and present] ***********************************
changed: [vm01]

TASK [Static website content is present] ***************************************
changed: [vm01]

TASK [Nginx configuration applied] *********************************************
changed: [vm01]

RUNNING HANDLER [Restart nginx] ************************************************
changed: [vm01]

PLAY RECAP ************************************************************

### Run Playbook with HTTPS/Firewall setup

#### Download packages using Ansible Galaxy

In [2]:
!ansible-galaxy install -r requirements.yml

Starting galaxy role install process
- downloading role 'firewall', owned by geerlingguy
- downloading role from https://github.com/geerlingguy/ansible-role-firewall/archive/2.5.1.tar.gz
- extracting geerlingguy.firewall to /home/kaveen/.ansible/roles/geerlingguy.firewall
- geerlingguy.firewall (2.5.1) was installed successfully
- downloading role 'certbot', owned by geerlingguy
- downloading role from https://github.com/geerlingguy/ansible-role-certbot/archive/5.1.0.tar.gz
- extracting geerlingguy.certbot to /home/kaveen/.ansible/roles/geerlingguy.certbot
- geerlingguy.certbot (5.1.0) was installed successfully


#### Run complete playbook

In [4]:
!ansible-playbook -i inventory.yaml playbook_3.yaml


PLAY [Configure Servers] *******************************************************

TASK [Gathering Facts] *********************************************************
ok: [vm01]

TASK [Apt cache is updated] ****************************************************
ok: [vm01]

TASK [Custom MOTD is set] ******************************************************
ok: [vm01]

PLAY [Configure Webservers] ****************************************************

TASK [Gathering Facts] *********************************************************
ok: [vm01]

TASK [geerlingguy.firewall : Ensure iptables is present.] **********************
changed: [vm01]

TASK [geerlingguy.firewall : Flush iptables the first time playbook runs.] *****
changed: [vm01]

TASK [geerlingguy.firewall : Copy firewall script into place.] *****************
changed: [vm01]

TASK [geerlingguy.firewall : Copy firewall init script into place.] ************
skipping: [vm01]

TASK [geerlingguy.firewall : Copy firewall systemd unit file into plac

### Nice to know

#### Validating changes using check mode

In [5]:
!ansible-playbook -i inventory.yaml playbook_3.yaml --check --diff


PLAY [Configure Servers] *******************************************************

TASK [Gathering Facts] *********************************************************
ok: [vm01]

TASK [Apt cache is updated] ****************************************************
changed: [vm01]

TASK [Custom MOTD is set] ******************************************************
--- before: /etc/motd
+++ after: /home/kaveen/code/kaveenr/gapstars-ansible-for-devs-talk/files/motd.txt
@@ -4,4 +4,4 @@
 /_/ |_/___/_//_/\__/\_, / /___/
                    /___/       
 Property of Ashey, don't mess with it.
-
+Don't mess with this!

changed: [vm01]

PLAY [Configure Webservers] ****************************************************

TASK [Gathering Facts] *********************************************************
ok: [vm01]

TASK [geerlingguy.firewall : Ensure iptables is present.] **********************
ok: [vm01]

TASK [geerlingguy.firewall : Flush iptables the first time playbook runs.] *****
ok: [vm01]

TASK [geerling

### Deploy just the source using tags

In [1]:
!ansible-playbook -i inventory.yaml playbook_3.yaml --tags source


PLAY [Configure Servers] *******************************************************

TASK [Gathering Facts] *********************************************************
ok: [vm01]

PLAY [Configure Webservers] ****************************************************

TASK [Gathering Facts] *********************************************************
ok: [vm01]

TASK [Static website content is present] ***************************************
changed: [vm01]

PLAY RECAP *********************************************************************
vm01                       : ok=3    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

